<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b6a5841b343fa0c8cd9c7de651b262d76a11b241cb1682f6fdd2fd3181aefd58
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-08 11:11:41
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: -67.53 K (-0.48%)
Current PnL: -21.20 L (-14.07%)
CY Booked + Current PnL: -9.76 L (-6.48%)
-------------------
Total profit:  1.30 L
Total loss:  -22.50 L
-------------------
Total Booked + Current PnL: 17.22 L (13.92%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.71 L (63.19%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,67.0,H-MC,14.14,117792.0,17288.0,11815.0,2.39,17.20,10.03,28.96,79.0,1.46,0.84,57.01,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.38,39.0,M-SC,1.18,83296.0,-17481.0,17575.0,-0.09,-17.35,21.10,0.09,245.0,-0.99,0.60,9.43,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.02,46.0,H-MC,3.02,188220.0,23550.0,19782.0,-1.57,14.30,10.51,26.31,89.0,1.19,1.35,23.43,X40N,NTT,AC
19,CIPLA,1662.74,-22.87,39.0,H-LC,3.81,205514.0,1014.0,22278.0,-0.87,0.50,10.84,11.39,10.0,0.05,1.47,9.49,X40N,ATH,PHARMA
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.29,205050.0,13308.0,24442.0,-0.38,6.94,11.92,19.69,99.0,0.54,1.47,14.21,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HONAUT,58357.33,-22.02,50.0,H-SC,2.14,109830.0,-18108.0,65239.0,2.51,-14.15,59.40,36.84,143.0,-0.28,0.79,12.44,X40N,ATH,ELECTRICAL
50,NATIONALUM,244.55,-41.18,67.0,H-MC,14.14,117792.0,17288.0,11815.0,2.39,17.20,10.03,28.96,79.0,1.46,0.84,57.01,MH,ATH,METALS
40,INFY,2275.00,-18.52,52.0,H-LC,8.01,316816.0,3928.0,167754.0,1.98,1.26,52.95,54.87,3.0,0.02,2.27,7.99,X40,BTT,IT
74,TCS,4406.34,-28.69,50.0,H-LC,12.18,284247.0,-61709.0,129958.0,1.69,-17.84,45.72,19.73,1.0,-0.47,2.03,4.69,X40,ATH,IT
48,LTIM,7230.20,-2.89,58.0,H-LC,2.44,240660.0,-5491.0,84688.0,1.50,-2.23,35.19,32.18,16.0,-0.06,1.72,33.01,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-42.40,39.0,H-SC,15.98,87280.0,-15220.0,55824.0,-2.49,-14.85,63.96,39.61,114.0,-0.27,0.62,6.60,AR,ATH,MISC
46,KPIGREEN,731.05,-10.94,26.0,H-SC,6.53,106294.0,-19003.0,77935.0,-2.36,-15.17,73.32,47.03,141.0,-0.24,0.76,33.09,MH,ATH,POWER
75,TITAGARH,1548.00,-3.20,54.0,H-SC,4.75,206796.0,-35381.0,146143.0,-2.18,-14.61,70.67,45.74,158.0,-0.24,1.48,34.50,XY24,NTT,ENGINEERING
10,BANDHANBNK,400.00,-13.31,45.0,H-SC,2.94,216155.0,-62404.0,319845.0,-1.92,-22.40,147.97,92.42,151.0,-0.20,1.55,24.92,XY24,NTT,BANKS
82,VALIANTORG,838.00,-295.26,33.0,H-SC,1.72,108460.0,-55145.0,176464.0,-1.85,-33.71,162.70,74.15,139.0,-0.31,0.78,39.03,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.57,45.0,H-MC,1.90,134888.0,-900.0,43650.0,-1.10,-0.66,32.36,31.48,107.0,-0.02,0.96,11.38,X40,ATH,INSURANCE
86,WIPRO,420.0,-13.79,47.0,M-LC,5.88,151230.0,285.0,109173.0,0.16,0.19,72.19,72.51,53.0,0.00,1.08,6.22,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,104.20,60.0,H-SC,13.43,129114.0,-13365.0,32885.0,-1.24,-9.38,25.47,13.70,163.0,-0.41,0.92,52.39,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,39.0,M-SC,1.18,83296.0,-17481.0,17575.0,-0.09,-17.35,21.10,0.09,245.0,-0.99,0.60,9.43,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,49.0,H-SC,2.01,221364.0,-48303.0,84627.0,-0.21,-17.91,38.23,13.47,138.0,-0.57,1.58,12.56,XY24,NTT,PAINTS
18,CERA,9475.0,-23.39,34.0,H-SC,1.61,139656.0,-36247.0,78263.0,-0.28,-20.61,56.04,23.89,149.0,-0.46,1.00,20.61,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-23.37,48.0,H-MC,6.44,103692.0,-28143.0,70365.0,1.26,-21.35,67.86,32.03,98.0,-0.40,0.74,16.14,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,53.0,M-MC,12.39,228819.0,3857.0,163789.0,0.20,1.71,71.58,74.53,192.0,0.02,1.64,34.52,XY24,BTT,STEEL
32,HINDZINC,730.22,30.43,66.0,M-LC,6.87,212758.0,7682.0,104890.0,-0.16,3.75,49.30,54.89,52.0,0.07,1.52,28.74,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.02,46.0,H-MC,3.02,188220.0,23550.0,19782.0,-1.57,14.30,10.51,26.31,89.0,1.19,1.35,23.43,X40N,NTT,AC
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.29,205050.0,13308.0,24442.0,-0.38,6.94,11.92,19.69,99.0,0.54,1.47,14.21,XY25,NTT,AC
17,CAMS,5211.76,-6.49,44.0,H-SC,1.73,106344.0,4340.0,39581.0,-0.07,4.25,37.22,43.06,122.0,0.11,0.76,22.10,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,67.0,H-MC,14.14,117792.0,17288.0,11815.0,2.39,17.20,10.03,28.96,79.0,1.46,0.84,57.01,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,61.0,H-MC,11.12,190758.0,10950.0,101464.0,-1.75,6.09,53.19,62.52,88.0,0.11,1.36,42.25,XR,NTT,BANKS
32,HINDZINC,730.22,30.43,66.0,M-LC,6.87,212758.0,7682.0,104890.0,-0.16,3.75,49.30,54.89,52.0,0.07,1.52,28.74,X5K,ATH,METALS
37,INDIAMART,4810.62,-55.20,34.0,H-SC,9.07,124852.0,1516.0,130108.0,0.39,1.23,104.21,106.72,119.0,0.01,0.89,24.89,AR,ATH,MISC
86,WIPRO,420.00,-13.79,47.0,M-LC,5.88,151230.0,285.0,109173.0,0.16,0.19,72.19,72.51,53.0,0.00,1.08,6.22,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.35,28.0,H-LC,10.20,276126.0,-39716.0,152366.0,-1.64,-12.57,55.18,35.67,5.0,-0.26,1.97,0.00,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.55,31.0,H-LC,5.71,205744.0,-46024.0,99230.0,-0.63,-18.28,48.23,21.13,27.0,-0.46,1.47,10.67,X40,ATH,PAINTS
31,HINDUNILVR,2922.00,-10.83,39.0,H-LC,8.00,250490.0,-6573.0,41707.0,-0.49,-2.56,16.65,13.67,24.0,-0.16,1.79,17.33,XY25,NTT,FMCG
19,CIPLA,1662.74,-22.87,39.0,H-LC,3.81,205514.0,1014.0,22278.0,-0.87,0.50,10.84,11.39,10.0,0.05,1.47,9.49,X40N,ATH,PHARMA
42,ITC,452.00,-39.61,40.0,H-LC,2.32,195356.0,-4782.0,25670.0,-0.07,-2.39,13.14,10.44,4.0,-0.19,1.40,3.04,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-0.04,53.0,H-LC,1.95,160555.0,-25540.0,73020.0,-1.22,-13.72,45.48,25.51,15.0,-0.35,1.15,18.32,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,40.0,H-LC,2.32,195356.0,-4782.0,25670.0,-0.07,-2.39,13.14,10.44,4.0,-0.19,1.40,3.04,X40,NTT,FMCG
48,LTIM,7230.2,-2.89,58.0,H-LC,2.44,240660.0,-5491.0,84688.0,1.50,-2.23,35.19,32.18,16.0,-0.06,1.72,33.01,X200,ATH,IT
51,NESTLEIND,1377.0,-11.15,47.0,H-LC,2.82,273569.0,8143.0,48641.0,-0.65,3.07,17.78,21.40,11.0,0.17,1.96,9.92,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.41,45.0,H-LC,3.21,214048.0,3682.0,25108.0,-0.92,1.75,11.73,13.68,37.0,0.15,1.53,18.56,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.35,28.0,H-LC,10.20,276126.0,-39716.0,152366.0,-1.64,-12.57,55.18,35.67,5.0,-0.26,1.97,0.00,X40N,ATH,FMCG
42,ITC,452.00,-39.61,40.0,H-LC,2.32,195356.0,-4782.0,25670.0,-0.07,-2.39,13.14,10.44,4.0,-0.19,1.40,3.04,X40,NTT,FMCG
74,TCS,4406.34,-28.69,50.0,H-LC,12.18,284247.0,-61709.0,129958.0,1.69,-17.84,45.72,19.73,1.0,-0.47,2.03,4.69,X40,ATH,IT
80,UNITDSPR,1644.00,-12.37,51.0,H-LC,6.77,229586.0,-4558.0,53172.0,-1.26,-1.95,23.16,20.77,86.0,-0.09,1.64,4.84,X40N,NTT,BREWERIES
1,ABB,7934.00,-40.55,44.0,H-LC,7.51,248544.0,-13075.0,132300.0,-0.80,-5.00,53.23,45.57,7.0,-0.10,1.78,5.76,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-0.04,53.0,H-LC,1.95,160555.0,-25540.0,73020.0,-1.22,-13.72,45.48,25.51,15.0,-0.35,1.15,18.32,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,40.0,H-LC,2.32,195356.0,-4782.0,25670.0,-0.07,-2.39,13.14,10.44,4.0,-0.19,1.40,3.04,X40,NTT,FMCG
19,CIPLA,1662.74,-22.87,39.0,H-LC,3.81,205514.0,1014.0,22278.0,-0.87,0.50,10.84,11.39,10.0,0.05,1.47,9.49,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-18.55,31.0,H-LC,5.71,205744.0,-46024.0,99230.0,-0.63,-18.28,48.23,21.13,27.0,-0.46,1.47,10.67,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,45.0,H-LC,3.21,214048.0,3682.0,25108.0,-0.92,1.75,11.73,13.68,37.0,0.15,1.53,18.56,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.69,50.0,H-LC,12.18,284247.0,-61709.0,129958.0,1.69,-17.84,45.72,19.73,1.0,-0.47,2.03,4.69,X40,ATH,IT
40,INFY,2275.00,-18.52,52.0,H-LC,8.01,316816.0,3928.0,167754.0,1.98,1.26,52.95,54.87,3.0,0.02,2.27,7.99,X40,BTT,IT
42,ITC,452.00,-39.61,40.0,H-LC,2.32,195356.0,-4782.0,25670.0,-0.07,-2.39,13.14,10.44,4.0,-0.19,1.40,3.04,X40,NTT,FMCG
83,VBL,671.64,-23.35,28.0,H-LC,10.20,276126.0,-39716.0,152366.0,-1.64,-12.57,55.18,35.67,5.0,-0.26,1.97,0.00,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,44.0,H-LC,7.51,248544.0,-13075.0,132300.0,-0.80,-5.00,53.23,45.57,7.0,-0.10,1.78,5.76,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,30.86,79738.0,-33811.0,73861.0,0.36,-29.78,92.63,35.27,268.0,-0.46,0.57,95.16,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,46.0,H-SC,10.90,92070.0,-5910.0,27511.0,-0.76,-6.03,29.88,22.05,152.0,-0.21,0.66,33.83,XR,ATH,FINANCE
50,NATIONALUM,244.55,-41.18,67.0,H-MC,14.14,117792.0,17288.0,11815.0,2.39,17.20,10.03,28.96,79.0,1.46,0.84,57.01,MH,ATH,METALS
78,UJJIVANSFB,60.00,104.20,60.0,H-SC,13.43,129114.0,-13365.0,32885.0,-1.24,-9.38,25.47,13.70,163.0,-0.41,0.92,52.39,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-182.95,65.0,M-SC,13.50,144270.0,-4950.0,149132.0,-1.71,-3.32,103.37,96.62,208.0,-0.03,1.03,56.13,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-182.95,65.0,M-SC,13.50,144270.0,-4950.0,149132.0,-1.71,-3.32,103.37,96.62,208.0,-0.03,1.03,56.13,XY25,NTT,FINANCE
69,SURYODAY,240.00,60.09,60.0,H-SC,7.33,147288.0,-31783.0,97505.0,-0.59,-17.75,66.20,36.71,135.0,-0.33,1.05,46.14,XR,NTT,BANKS
26,GREENPANEL,537.00,233.98,60.0,M-SC,5.08,148986.0,-34092.0,113065.0,-0.23,-18.62,75.89,43.14,230.0,-0.30,1.07,40.14,XY24,NTT,MISC
32,HINDZINC,730.22,30.43,66.0,M-LC,6.87,212758.0,7682.0,104890.0,-0.16,3.75,49.30,54.89,52.0,0.07,1.52,28.74,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,61.0,H-MC,11.12,190758.0,10950.0,101464.0,-1.75,6.09,53.19,62.52,88.0,0.11,1.36,42.25,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.12
1,25,43.42
2,50,75.09


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.08
LC    32.20
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.05
X40      14.67
X40N     11.16
XR       10.95
XY25     10.38
AR        8.14
OX40N     7.69
X200      1.72
MH        1.60
X5K       1.52
SR        1.13
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.97
H-LC    25.68
H-MC    20.47
M-SC    13.69
M-LC     5.50
M-MC     2.96
L-SC     1.42
L-LC     1.02
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.72,-7.04,42.31
IT,12.79,-17.56,81.23
FINANCE,10.53,-14.50,69.79
MISC,6.90,-19.01,84.94
BANKS,6.29,-13.27,72.86
PAINTS,5.53,-19.41,37.20
ELECTRICAL,5.40,-9.33,47.69
INSURANCE,3.81,-3.72,42.49
AC,3.51,1.21,15.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3236001.0,22
XR,1379808.0,14
AR,1192560.0,9
X40,789046.0,10
X40N,673592.0,9
OX40N,570443.0,10
XY25,487188.0,7
SR,262795.0,2
X5K,104890.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3740834.0,29
M-SC,1520729.0,17
H-MC,1291450.0,15
H-LC,1202871.0,15
M-LC,380947.0,4
M-MC,370094.0,2
L-SC,261746.0,3
L-MC,60194.0,1
L-LC,41896.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1202639.0      6
M-SC       XY24       851618.0      7
H-SC       AR         818044.0      5
           XR         805298.0      7
H-MC       XY24       575149.0      4
H-LC       X40        499974.0      5
M-MC       XY24       370094.0      2
H-SC       X40N       309302.0      4
           OX40N      264821.0      4
           SR         262795.0      2
H-LC       X40N       227816.0      3
H-MC       X40        215479.0      4
H-LC       AR         205320.0      2
H-MC       XY25       180704.0      2
L-SC       XR         172583.0      2
M-SC       AR         169196.0      2
M-LC       XY24       166884.0      2
M-SC       XY25       149132.0      1
           OX40N      146094.0      4
H-MC       X40N       136474.0      2
M-SC       XR         131096.0      2
H-LC       XY25       115456.0      3
M-LC       XR         109173.0      1
           X5K        104890.0      1
H-MC       XR         101464.0      1
L-SC       OX40N       89163.0      1
H-LC       X200        84688.0      1
H-SC       MH          77935.0      1
M-SC       X40         73593.0      1
H-MC       OX40N       70365.0      1
H-LC       XY24        69617.0      1
L-MC       XR          60194.0      1
L-LC       XY25        41896.0      1
H-MC       MH          11815.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
